LLAMA INDEX 2

In [1]:
!pip install pypdf

In [2]:
!pip install transformers
!pip install einops
!pip install langchain
!pip install accelerate
!pip install bitsandbytes

In [3]:
!pip install install sentence_transformers

In [4]:
!pip install llama_index

In [5]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/data").load_data()
documents

[Document(id_='938cb4eb-b76f-4a73-9a40-fd0fa1ed8b30', embedding=None, metadata={'page_label': '1', 'file_name': 'FINAL REPORT.pdf', 'file_path': '/content/drive/MyDrive/data/FINAL REPORT.pdf', 'file_type': 'application/pdf', 'file_size': 346260, 'creation_date': '2024-02-07', 'last_modified_date': '2023-10-26', 'last_accessed_date': '2024-02-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='  1   C NN  AND CLASSICAL DEEP  LEARNING \nCLASSIFIERS  TO IDENTIFY MALIGNANT \nMELANOMA  \n  \nDR. A.R.ARUNARANI  \nASSISTANT PROFESSOR  \nDEPARTMENT OF  \nCOMPUTATIONAL INTELLIGENCE \nSRM INSTITUTE OF SCIENCE AND \nTECHNOLOGY.  R.AKASH  \nSTUDENT  \nDEPARTMENT OF  \nCOMPUTATIONAL INTELLIGENCE \nSRM INSTITUTE OF SCIENCE AND \nTECHNOLOGY .  N. VIVEK  \nSTUDENT  \n

In [8]:
system_prompt="""
You are a Q&A assistant.Your goal is to answer as
accurately as possible based in the instructions and contex provided.
"""
## format for llama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [9]:
!!huggingface-cli login

['',
 '    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|',
 '    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|',
 '    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|',
 '    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|',
 '    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|',
 '',
 '    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.',
 '    Setting a new token will erase the existing one.',
 '    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .',
 'Token: ',
 'Add token as git credential? (Y/n) y',
 'Token is valid (permiss

In [10]:
import torch
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embd_model=LangchainEmbedding(
    HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


In [12]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embd_model
)

In [13]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7c83d13d7670>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7c83d13d7670>, id_func=<function default_id_func at 0x7c84866bfd90>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7c8485764520>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7c83d13d7670>)

In [14]:
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

In [15]:
index

In [17]:
query_engine=index.as_query_engine()

In [31]:
response=query_engine.query("what is cnn?")
print(response)

CNN stands for Convolutional Neural Network. It is a type of deep learning model that is commonly used for image recognition tasks, including the diagnosis of malignant melanoma from dermoscopic images. In a CNN, visual features are extracted from images using a series of convolutional and pooling layers, followed by fully connected layers for classification. The convolutional layers are responsible for detecting patterns and features within the images, while the fully connected layers make the final predictions based on these features. CNNs have shown great promise in this field due to their ability to automatically learn hierarchical representations of images, making them a powerful tool for melanoma detection.
